In [1]:
from pyspark.sql import SparkSession

spark = SparkSession \
    .builder \
    .appName("Python Spark regression example") \
    .config("spark.some.config.option", "some-value") \
    .getOrCreate()

In [2]:
drop_name = ['_c0']
df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').\
            load("Advertising.csv",header=True).drop(*drop_name)
df.show(5,True)
df.printSchema()

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)



In [3]:
df.describe().show()

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|               0.0|               0.3|               1.6|
|    max|            296.4|              49.6|             114.0|              27.0|
+-------+-----------------+------------------+------------------+------------------+



In [4]:
# Convert the data to dense vector (features and label)
from pyspark.sql import Row
from pyspark.ml.linalg import Vectors

# I provide two ways to build the features and labels

# method 1 (good for small feature):
#def transData(row):
#    return Row(label=row["Sales"],
#               features=Vectors.dense([row["TV"],
#                                       row["Radio"],
#                                       row["Newspaper"]]))

# Method 2 (good for large features):
def transData(data):
    return data.rdd.map(lambda r: [Vectors.dense(r[:-1]),r[-1]]).toDF(['features','label'])

In [6]:
# define get_dummy
def get_dummy(df,indexCol,categoricalCols,continuousCols,labelCol,dropLast=False):

    '''
    Get dummy variables and concat with continuous variables for ml modeling.
    :param df: the dataframe
    :param categoricalCols: the name list of the categorical data
    :param continuousCols:  the name list of the numerical data
    :param labelCol:  the name of label column
    :param dropLast:  the flag of drop last column
    :return: feature matrix

    :author: Wenqiang Feng
    :email:  von198@gmail.com

    >>> df = spark.createDataFrame([
                  (0, "a",3.5,30),
                  (1, "b",5.7,50),
                  (2, "c",20,45.7),
                  (3, "a", 8.2,25.3),
                  (4, "a",9.1,32),
                  (5, "c",2.5,19.4)
              ], ["id", "category","cont",'price'])

    >>> indexCol = 'id'
    >>> categoricalCols = ['category']
    >>> continuousCols = ['cont']
    >>> labelCol = []  or labelCol=['price']

    >>> mat = get_dummy(df,indexCol,categoricalCols,continuousCols,labelCol)
    >>> mat.show()

    >>>
        +---+-------------+
        | id|     features|
        +---+-------------+
        |  0|[1.0,0.0,0.0]|
        |  1|[0.0,0.0,1.0]|
        |  2|[0.0,1.0,0.0]|
        |  3|[1.0,0.0,0.0]|
        |  4|[1.0,0.0,0.0]|
        |  5|[0.0,1.0,0.0]|
        +---+-------------+
    '''

    from pyspark.ml import Pipeline
    from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorAssembler
    from pyspark.sql.functions import col

    indexers = [ StringIndexer(inputCol=c, outputCol="{0}_indexed".format(c))
                 for c in categoricalCols ]

    # default setting: dropLast=True
    encoders = [ OneHotEncoder(inputCol=indexer.getOutputCol(),
                 outputCol="{0}_encoded".format(indexer.getOutputCol()),dropLast=dropLast)
                 for indexer in indexers ]

    assembler = VectorAssembler(inputCols=[encoder.getOutputCol() for encoder in encoders]
                                + continuousCols, outputCol="features")

    pipeline = Pipeline(stages=indexers + encoders + [assembler])

    model=pipeline.fit(df)
    data = model.transform(df)

    if indexCol and labelCol:
        # for supervised learning
        data = data.withColumn('label',col(labelCol))
        return data.select(indexCol,'features','label')
    elif not indexCol and labelCol:
        # for supervised learning
        data = data.withColumn('label',col(labelCol))
        return data.select('features','label')
    elif indexCol and not labelCol:
        # for unsupervised learning
        return data.select(indexCol,'features')
    elif not indexCol and not labelCol:
        # for unsupervised learning
        return data.select('features')

In [37]:
mySchema = StructType([StructField("id", IntegerType(), True)\
                       ,StructField("cont", DoubleType(), True)\
                       ,StructField("category", StringType(), True)\
                       ,StructField("price", DoubleType(), True)])

In [61]:
df1 = spark.createDataFrame([
                  (0, 3.5, "a",30.5),
                  (1, 5.0, "b", 49.9),
                  (2, 8.5, "c", 82.4),
                  (3, 15.0, "a", 13.0),
                  (4, 20.4, "a", 18.0),
                  (5, 15.3, "c", 12.3)], 
                   ["id", "cont", "category","price"],mySchema)

In [65]:
df1.show()

+---+----+--------+-----+
| id|cont|category|price|
+---+----+--------+-----+
|  0| 3.5|       a| 30.5|
|  1| 5.0|       b| 49.9|
|  2| 8.5|       c| 82.4|
|  3|15.0|       a| 13.0|
|  4|20.4|       a| 18.0|
|  5|15.3|       c| 12.3|
+---+----+--------+-----+



In [7]:
#example for get_dummy
from pyspark.sql.types import *

mySchema = StructType([StructField("id", IntegerType(), True)\
                       ,StructField("cont", DoubleType(), True)\
                       ,StructField("category", StringType(), True)\
                       ,StructField("price", DoubleType(), True)])
df1 = spark.createDataFrame([
                  (0, 3.5, "a",30.5),
                  (1, 5.0, "b", 49.9),
                  (2, 8.5, "c", 82.4),
                  (3, 15.0, "a", 13.0),
                  (4, 20.4, "a", 18.0),
                  (5, 15.3, "c", 12.3)], 
                   ["id", "cont", "category","price"],mySchema)
indexCol = 'id'
categoricalCols = ['category']
continuousCols = ['cont',"price"]
labelCol = []
#labelCol=['price']
mat = get_dummy(df1,indexCol,categoricalCols,continuousCols,labelCol)
mat.show()

+---+--------------------+
| id|            features|
+---+--------------------+
|  0|[1.0,0.0,0.0,3.5,...|
|  1|[0.0,0.0,1.0,5.0,...|
|  2|[0.0,1.0,0.0,8.5,...|
|  3|[1.0,0.0,0.0,15.0...|
|  4|[1.0,0.0,0.0,20.4...|
|  5|[0.0,1.0,0.0,15.3...|
+---+--------------------+



In [8]:
# Transform the dataset to DataFrame
transformed= transData(df)
transformed.show(5)

+-----------------+-----+
|         features|label|
+-----------------+-----+
|[230.1,37.8,69.2]| 22.1|
| [44.5,39.3,45.1]| 10.4|
| [17.2,45.9,69.3]|  9.3|
|[151.5,41.3,58.5]| 18.5|
|[180.8,10.8,58.4]| 12.9|
+-----------------+-----+
only showing top 5 rows



In [14]:
from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=["TV", "Radio", "Newspaper"],outputCol="features")
# Now let us use the transform method to transform our dataset
assembler.transform(df).show()

+-----+-----+---------+-----+-----------------+
|   TV|Radio|Newspaper|Sales|         features|
+-----+-----+---------+-----+-----------------+
|230.1| 37.8|     69.2| 22.1|[230.1,37.8,69.2]|
| 44.5| 39.3|     45.1| 10.4| [44.5,39.3,45.1]|
| 17.2| 45.9|     69.3|  9.3| [17.2,45.9,69.3]|
|151.5| 41.3|     58.5| 18.5|[151.5,41.3,58.5]|
|180.8| 10.8|     58.4| 12.9|[180.8,10.8,58.4]|
|  8.7| 48.9|     75.0|  7.2|  [8.7,48.9,75.0]|
| 57.5| 32.8|     23.5| 11.8| [57.5,32.8,23.5]|
|120.2| 19.6|     11.6| 13.2|[120.2,19.6,11.6]|
|  8.6|  2.1|      1.0|  4.8|    [8.6,2.1,1.0]|
|199.8|  2.6|     21.2| 10.6| [199.8,2.6,21.2]|
| 66.1|  5.8|     24.2|  8.6|  [66.1,5.8,24.2]|
|214.7| 24.0|      4.0| 17.4| [214.7,24.0,4.0]|
| 23.8| 35.1|     65.9|  9.2| [23.8,35.1,65.9]|
| 97.5|  7.6|      7.2|  9.7|   [97.5,7.6,7.2]|
|204.1| 32.9|     46.0| 19.0|[204.1,32.9,46.0]|
|195.4| 47.7|     52.9| 22.4|[195.4,47.7,52.9]|
| 67.8| 36.6|    114.0| 12.5|[67.8,36.6,114.0]|
|281.4| 39.6|     55.8| 24.4|[281.4,39.6

In [15]:
df.show()

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
|  8.7| 48.9|     75.0|  7.2|
| 57.5| 32.8|     23.5| 11.8|
|120.2| 19.6|     11.6| 13.2|
|  8.6|  2.1|      1.0|  4.8|
|199.8|  2.6|     21.2| 10.6|
| 66.1|  5.8|     24.2|  8.6|
|214.7| 24.0|      4.0| 17.4|
| 23.8| 35.1|     65.9|  9.2|
| 97.5|  7.6|      7.2|  9.7|
|204.1| 32.9|     46.0| 19.0|
|195.4| 47.7|     52.9| 22.4|
| 67.8| 36.6|    114.0| 12.5|
|281.4| 39.6|     55.8| 24.4|
| 69.2| 20.5|     18.3| 11.3|
|147.3| 23.9|     19.1| 14.6|
+-----+-----+---------+-----+
only showing top 20 rows



In [76]:
df2=df1.drop('id')
df2.show()

+----+--------+-----+
|cont|category|price|
+----+--------+-----+
| 3.5|       a| 30.5|
| 5.0|       b| 49.9|
| 8.5|       c| 82.4|
|15.0|       a| 13.0|
|20.4|       a| 18.0|
|15.3|       c| 12.3|
+----+--------+-----+



In [9]:
# Deal With Categorical Variables
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator

# Automatically identify categorical features, and index them.
# We specify maxCategories so features with > 4 distinct values are treated as continuous.

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

data = featureIndexer.transform(transformed)
data.show(5)

+-----------------+-----+-----------------+
|         features|label|  indexedFeatures|
+-----------------+-----+-----------------+
|[230.1,37.8,69.2]| 22.1|[230.1,37.8,69.2]|
| [44.5,39.3,45.1]| 10.4| [44.5,39.3,45.1]|
| [17.2,45.9,69.3]|  9.3| [17.2,45.9,69.3]|
|[151.5,41.3,58.5]| 18.5|[151.5,41.3,58.5]|
|[180.8,10.8,58.4]| 12.9|[180.8,10.8,58.4]|
+-----------------+-----+-----------------+
only showing top 5 rows



In [85]:
#df3 = spark.createDataFrame([(Vectors.dense([-1.0, 0.0]),),
#                             (Vectors.dense([0.0, 1.0]),), 
#                             (Vectors.dense([0.0, 2.0]),)], ["a"])
indexer = VectorIndexer(maxCategories=2, inputCol="a", outputCol="indexed")
model = indexer.fit(df3)
model.transform(df3).head().indexed
model.categoryMaps

{0: {0.0: 0, -1.0: 1}}

{0: {0.0: 0, -1.0: 1}}

In [10]:
# Split the data into training and test sets
(trainingData, testData) = transformed.randomSplit([0.7, 0.3])
trainingData.show(5)
testData.show(5)

+---------------+-----+
|       features|label|
+---------------+-----+
| [4.1,11.6,5.7]|  3.2|
| [5.4,29.9,9.4]|  5.3|
|[7.3,28.1,41.4]|  5.5|
|[7.8,38.9,50.6]|  6.6|
| [8.4,27.2,2.1]|  5.7|
+---------------+-----+
only showing top 5 rows

+----------------+-----+
|        features|label|
+----------------+-----+
|  [0.7,39.6,8.7]|  1.6|
|   [8.6,2.1,1.0]|  4.8|
|[11.7,36.9,45.2]|  7.3|
|[13.2,15.9,49.6]|  5.6|
|[18.7,12.1,23.4]|  6.7|
+----------------+-----+
only showing top 5 rows



In [87]:
# Import LinearRegression class
from pyspark.ml.regression import LinearRegression

# Define LinearRegression algorithm
lr = LinearRegression()

In [88]:
# Chain indexer and lr in a Pipeline
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[featureIndexer, lr])

model = pipeline.fit(trainingData)

In [91]:
#summary of the model
def modelsummary(model):
    import numpy as np
    print ("Note: the last rows are the information for Intercept")
    print ("##","-------------------------------------------------")
    print ("##","  Estimate   |   Std.Error | t Values  |  P-value")
    coef = np.append(list(model.coefficients),model.intercept)
    Summary=model.summary

    for i in range(len(Summary.pValues)):
        print ("##",'{:10.6f}'.format(coef[i]),\
        '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
        '{:8.3f}'.format(Summary.tValues[i]),\
        '{:10.6f}'.format(Summary.pValues[i]))

    print ("##",'---')
    print ("##","Mean squared error: % .6f" \
           % Summary.meanSquaredError, ", RMSE: % .6f" \
           % Summary.rootMeanSquaredError )
    print ("##","Multiple R-squared: %f" % Summary.r2, ", \
            Total iterations: %i"% Summary.totalIterations)
    
modelsummary(model.stages[-1])

Note: the last rows are the information for Intercept
## -------------------------------------------------
##   Estimate   |   Std.Error | t Values  |  P-value
##   0.046021   0.001679   27.414   0.000000
##   0.180169   0.010896   16.535   0.000000
##   0.007268   0.007665    0.948   0.344691
##   2.835365   0.370348    7.656   0.000000
## ---
## Mean squared error:  2.991112 , RMSE:  1.729483
## Multiple R-squared: 0.896211 ,             Total iterations: 1


In [92]:
# Make predictions.
predictions = model.transform(testData)

# Select example rows to display.
predictions.select("features","label","prediction").show(5)

+----------------+-----+------------------+
|        features|label|        prediction|
+----------------+-----+------------------+
| [7.8,38.9,50.6]|  6.6| 10.57065065502108|
|[11.7,36.9,45.2]|  7.3|10.350549256693258|
|[16.9,43.7,89.4]|  8.7|12.136236800697118|
| [17.2,4.1,31.6]|  5.9| 4.595276736599125|
|[18.7,12.1,23.4]|  6.7| 6.046067314265413|
+----------------+-----+------------------+
only showing top 5 rows



In [93]:
# Evaluation
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

# calculate R2
y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {0}'.format(r2_score))

Root Mean Squared Error (RMSE) on test data = 1.54934
r2_score: 0.8946265735228226


## Generalized linear regression

In [11]:
# with same Advertising data
drop_name = ['_c0']
df = spark.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').\
            load("Advertising.csv",header=True).drop(*drop_name)
df.show(5,True)
df.printSchema()
df.describe().show()

transformed= transData(df)
transformed.show(5)

featureIndexer = VectorIndexer(inputCol="features", \
                               outputCol="indexedFeatures",\
                               maxCategories=4).fit(transformed)

data = featureIndexer.transform(transformed)
data.show(5)

+-----+-----+---------+-----+
|   TV|Radio|Newspaper|Sales|
+-----+-----+---------+-----+
|230.1| 37.8|     69.2| 22.1|
| 44.5| 39.3|     45.1| 10.4|
| 17.2| 45.9|     69.3|  9.3|
|151.5| 41.3|     58.5| 18.5|
|180.8| 10.8|     58.4| 12.9|
+-----+-----+---------+-----+
only showing top 5 rows

root
 |-- TV: double (nullable = true)
 |-- Radio: double (nullable = true)
 |-- Newspaper: double (nullable = true)
 |-- Sales: double (nullable = true)

+-------+-----------------+------------------+------------------+------------------+
|summary|               TV|             Radio|         Newspaper|             Sales|
+-------+-----------------+------------------+------------------+------------------+
|  count|              200|               200|               200|               200|
|   mean|         147.0425|23.264000000000024|30.553999999999995|14.022500000000003|
| stddev|85.85423631490805|14.846809176168728| 21.77862083852283| 5.217456565710477|
|    min|              0.7|             

In [12]:
# use same train and test data
(trainingData, testData) = transformed.randomSplit([0.7, 0.3])
#(trainingData, testData) = data.randomSplit([0.7, 0.3])
trainingData.show(5)
testData.show(5)

+---------------+-----+
|       features|label|
+---------------+-----+
| [0.7,39.6,8.7]|  1.6|
| [4.1,11.6,5.7]|  3.2|
|[7.8,38.9,50.6]|  6.6|
| [8.4,27.2,2.1]|  5.7|
|  [8.6,2.1,1.0]|  4.8|
+---------------+-----+
only showing top 5 rows

+----------------+-----+
|        features|label|
+----------------+-----+
|  [5.4,29.9,9.4]|  5.3|
| [7.3,28.1,41.4]|  5.5|
|[16.9,43.7,89.4]|  8.7|
|[18.8,21.7,50.4]|  7.0|
|[25.1,25.7,43.3]|  8.5|
+----------------+-----+
only showing top 5 rows



In [57]:
trainingData.collect()[0:5]

[Row(features=DenseVector([0.7, 39.6, 8.7]), label=1.6, indexedFeatures=DenseVector([0.7, 39.6, 8.7])),
 Row(features=DenseVector([5.4, 29.9, 9.4]), label=5.3, indexedFeatures=DenseVector([5.4, 29.9, 9.4])),
 Row(features=DenseVector([7.8, 38.9, 50.6]), label=6.6, indexedFeatures=DenseVector([7.8, 38.9, 50.6])),
 Row(features=DenseVector([8.4, 27.2, 2.1]), label=5.7, indexedFeatures=DenseVector([8.4, 27.2, 2.1])),
 Row(features=DenseVector([8.6, 2.1, 1.0]), label=4.8, indexedFeatures=DenseVector([8.6, 2.1, 1.0]))]

In [97]:
# Import LinearRegression class
from pyspark.ml.regression import GeneralizedLinearRegression

# Define LinearRegression algorithm
glr = GeneralizedLinearRegression(family="gaussian", link="identity",\
                                  maxIter=10, regParam=0.3)

In [98]:
# Chain indexer and tree in a Pipeline
pipeline = Pipeline(stages=[featureIndexer, glr])

model = pipeline.fit(trainingData)

In [99]:
def modelsummary(model):
    import numpy as np
    print ("Note: the last rows are the information for Intercept")
    print ("##","-------------------------------------------------")
    print ("##","  Estimate   |   Std.Error | t Values  |  P-value")
    coef = np.append(list(model.coefficients),model.intercept)
    Summary=model.summary

    for i in range(len(Summary.pValues)):
        print ("##",'{:10.6f}'.format(coef[i]),\
        '{:10.6f}'.format(Summary.coefficientStandardErrors[i]),\
        '{:8.3f}'.format(Summary.tValues[i]),\
        '{:10.6f}'.format(Summary.pValues[i]))

    print ("##",'---')
#     print ("##","Mean squared error: % .6f" \
#            % Summary.meanSquaredError, ", RMSE: % .6f" \
#            % Summary.rootMeanSquaredError )
#     print ("##","Multiple R-squared: %f" % Summary.r2, ", \
#             Total iterations: %i"% Summary.totalIterations)


modelsummary(model.stages[-1])

Note: the last rows are the information for Intercept
## -------------------------------------------------
##   Estimate   |   Std.Error | t Values  |  P-value
##   0.044130   0.001685   26.183   0.000000
##   0.176380   0.010259   17.193   0.000000
##   0.000266   0.006635    0.040   0.968110
##   3.486282   0.375172    9.292   0.000000
## ---


In [100]:
# Make predictions.
predictions = model.transform(testData)
predictions.select("features","label","prediction").show(5)

+----------------+-----+------------------+
|        features|label|        prediction|
+----------------+-----+------------------+
|  [4.1,11.6,5.7]|  3.2| 5.714736129608138|
|   [8.6,2.1,1.0]|  4.8|  4.23646503576621|
| [8.7,48.9,75.0]|  7.2|12.515118025391684|
|[11.7,36.9,45.2]|  7.3| 10.52303137780692|
|[18.8,21.7,50.4]|  7.0|  8.15676524189303|
+----------------+-----+------------------+
only showing top 5 rows



In [101]:
# evaluation
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.evaluation import RegressionEvaluator
# Select (prediction, true label) and compute test error
evaluator = RegressionEvaluator(labelCol="label",
                                predictionCol="prediction",
                                metricName="rmse")

rmse = evaluator.evaluate(predictions)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

#R2
y_true = predictions.select("label").toPandas()
y_pred = predictions.select("prediction").toPandas()

import sklearn.metrics
r2_score = sklearn.metrics.r2_score(y_true, y_pred)
print('r2_score: {0}'.format(r2_score))

Root Mean Squared Error (RMSE) on test data = 1.67846
r2_score: 0.8911232952954501


## example with power plant data

In [16]:
# https://blog.epigno.systems/2018/02/18/machine-learning-with-pyspark-linear-regression/
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import *

In [17]:
# File location and type
file_location = "pp_data.csv"
file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
data = spark.read.format(file_type) \
  .option("inferSchema", infer_schema) \
  .option("header", first_row_is_header) \
  .option("sep", delimiter) \
  .load(file_location)

data.show(5)

+-----+-----+-------+-----+------+
|   AT|    V|     AP|   RH|    PE|
+-----+-----+-------+-----+------+
|14.96|41.76|1024.07|73.17|463.26|
|25.18|62.96|1020.04|59.08|444.37|
| 5.11| 39.4|1012.16|92.14|488.56|
|20.86|57.32|1010.24|76.64|446.48|
|10.82| 37.5|1009.23|96.62| 473.9|
+-----+-----+-------+-----+------+
only showing top 5 rows



In [122]:
data.describe().show()

+-------+------------------+------------------+------------------+-----------------+------------------+
|summary|                AT|                 V|                AP|               RH|                PE|
+-------+------------------+------------------+------------------+-----------------+------------------+
|  count|             47840|             47840|             47840|            47840|             47840|
|   mean|19.651231187290833| 54.30580372073524|1013.2590781772577| 73.3089778428094|454.36500940635983|
| stddev|7.4521616583400085|12.707361709685806| 5.938535418520848|14.59965835208147|17.066281466837733|
|    min|              1.81|             25.36|            992.89|            25.56|            420.26|
|    max|             37.11|             81.56|            1033.3|           100.16|            495.76|
+-------+------------------+------------------+------------------+-----------------+------------------+



In [18]:
#features = ["temperature", "exhaust_vacuum", "ambient_pressure", "relative_humidity"]
from pyspark.sql.functions import col, abs
features = ["AT", "V", "AP", "RH"]
lr_data = data.select(col("PE").alias("label"), *features)
lr_data.printSchema()

root
 |-- label: double (nullable = true)
 |-- AT: double (nullable = true)
 |-- V: double (nullable = true)
 |-- AP: double (nullable = true)
 |-- RH: double (nullable = true)



In [19]:
lr_data.show()

+------+-----+-----+-------+-----+
| label|   AT|    V|     AP|   RH|
+------+-----+-----+-------+-----+
|463.26|14.96|41.76|1024.07|73.17|
|444.37|25.18|62.96|1020.04|59.08|
|488.56| 5.11| 39.4|1012.16|92.14|
|446.48|20.86|57.32|1010.24|76.64|
| 473.9|10.82| 37.5|1009.23|96.62|
|443.67|26.27|59.44|1012.23|58.77|
|467.35|15.89|43.96|1014.02|75.24|
|478.42| 9.48|44.71|1019.12|66.43|
|475.98|14.64| 45.0|1021.78|41.25|
| 477.5|11.74|43.56|1015.14|70.72|
|453.02|17.99|43.72|1008.64|75.04|
|453.99|20.14|46.93|1014.66|64.22|
|440.29|24.34| 73.5|1011.31|84.15|
|451.28|25.71|58.59|1012.77|61.83|
|433.99|26.19|69.34|1009.48|87.59|
|462.19|21.42|43.79|1015.76|43.08|
|467.54|18.21| 45.0|1022.86|48.84|
| 477.2|11.04|41.74| 1022.6|77.51|
|459.85|14.45|52.75|1023.97|63.59|
| 464.3|13.97|38.47|1015.15|55.28|
+------+-----+-----+-------+-----+
only showing top 20 rows



In [77]:
(training, test) = lr_data.randomSplit([.7, .3])

In [116]:
"""
This step is the beautiful part about Spark. You can build your ML pipeline and Spark will do all the heavy lifting for you.
This is how things work in our case:
we put all features into a vector
since we are dealing with numerical data, we scale those features
we chose the algorithm (in our case is linear regression)
and we create the pipeline with the steps and in the order mentioned above
"""
vectorizer = VectorAssembler(inputCols=features, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")
lr = LinearRegression(maxIter=10, regParam=.01)
print(lr.explainParams())
stages = [vectorizer, standardScaler, lr]
pipeline = Pipeline(stages=stages)

aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
epsilon: The shape parameter to control the amount of robustness. Must be > 1.0. Only valid when loss is huber (default: 1.35)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
loss: The loss function to be optimized. Supported options: squaredError, huber. (default: squaredError)
maxIter: max number of iterations (>= 0). (default: 100, current: 10)
predictionCol: prediction column name. (default: prediction)
regParam: regularization parameter (>= 0). (default: 0.0, current: 0.01)
solver: The solver algorithm for optimization. Supported options: auto, normal, l-bfgs. (default: auto)
standardization: whether to standardize the training featur

In [117]:
model = pipeline.fit(training)
pred = model.transform(test)
pred.show()

+------+-----+-----+-------+-----+--------------------+--------------------+------------------+
| label|   AT|    V|     AP|   RH|   unscaled_features|            features|        prediction|
+------+-----+-----+-------+-----+--------------------+--------------------+------------------+
|420.26|24.27|63.87|1018.88|53.96|[24.27,63.87,1018...|[3.25621100274884...| 446.3474964605456|
|420.26|24.27|63.87|1018.88|53.96|[24.27,63.87,1018...|[3.25621100274884...| 446.3474964605456|
|425.11|32.56|68.14|1004.02|35.04|[32.56,68.14,1004...|[4.36844788831901...|  431.045327024459|
|425.12|31.74|72.58|1007.26|59.58|[31.74,72.58,1007...|[4.25843169457142...|427.97721413740186|
|425.12|31.74|72.58|1007.26|59.58|[31.74,72.58,1007...|[4.25843169457142...|427.97721413740186|
|425.14|29.67|71.98|1005.16|67.75|[29.67,71.98,1005...|[3.98070788840372...|430.79761540492075|
|425.16|30.71|71.85|1008.07|72.05|[30.71,71.85,1008...|[4.12024062193725...|428.28306586751086|
|425.16|30.71|71.85|1008.07|72.05|[30.71

In [119]:
# The intercept is as follows:
intercept = model.stages[2].intercept
print(intercept)
# The coefficents (i.e., weights) are as follows:
weights = model.stages[2].coefficients
print(weights)

455.9505007439303
[-14.69528846198947,-3.0050825088904434,0.35993671509112307,-2.2748206161129683]


In [135]:
# Create a list of the column names (without PE)
featuresNoLabel = [col for col in data.columns if col != "PE"]
print(featuresNoLabel)
# Merge the weights and labels
coefficents = zip(weights, featuresNoLabel)
print(coefficents)
# Now let's sort the coefficients from greatest absolute weight most to the least absolute weight
#coefficents.sort(key=lambda tup: abs(tup[0]), reverse=True)

equation = "y = {intercept}".format(intercept=intercept)
variables = []
for x in coefficents:
    #print(x[0],x[1])
    #weight = abs(x[0])
    weight = x[0]
    name = x[1]
    #symbol = "+" if (x[0] > 0) else "-"
    equation += (" {} * {}".format(weight, name))

# Finally here is our equation
print("Linear Regression Equation: " + equation)

['AT', 'V', 'AP', 'RH']
Linear Regression Equation: y = 455.9505007439303 -14.69528846198947 * AT -3.0050825088904434 * V 0.35993671509112307 * AP -2.2748206161129683 * RH


In [100]:
#Evaluation
from pyspark.ml.evaluation import RegressionEvaluator
eval = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

# Root Mean Square Error
rmse = eval.evaluate(pred)
print("RMSE: %.3f" % rmse)

# Mean Square Error
mse = eval.evaluate(pred, {eval.metricName: "mse"})
print("MSE: %.3f" % mse)

# Mean Absolute Error
mae = eval.evaluate(pred, {eval.metricName: "mae"})
print("MAE: %.3f" % mae)

# r2 - coefficient of determination
r2 = eval.evaluate(pred, {eval.metricName: "r2"})
print("r2: %.3f" %r2)

RMSE: 4.524
MSE: 20.465
MAE: 3.612
r2: 0.929


## Another liear regression example

In [102]:
# https://medium.com/kharpann/perform-linear-regression-on-big-data-using-python-spark-and-mllib-b1204769547e
from pyspark.sql import SparkSession
from pyspark.ml.regression import LinearRegression

spark = SparkSession.builder.appName('lrex').getOrCreate()


In [109]:
# Load training data
data = spark.read.format("libsvm").load("./LinearRegression/sample_linear_regression_data.txt")
# Split the data
split_data = data.randomSplit([0.8,0.2])
train_data,test_data = split_data
training_data.show(5)

+-------------------+--------------------+
|              label|            features|
+-------------------+--------------------+
| -9.490009878824548|(10,[0,1,2,3,4,5,...|
| 0.2577820163584905|(10,[0,1,2,3,4,5,...|
| -4.438869807456516|(10,[0,1,2,3,4,5,...|
|-19.782762789614537|(10,[0,1,2,3,4,5,...|
| -7.966593841555266|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 5 rows



In [110]:
lin_reg=LinearRegression(featuresCol='features',labelCol='label',predictionCol='prediction') 
lin_reg_model=lin_reg.fit(training_data)
print(lin_reg_model.coefficients)
print(lin_reg_model.intercept)

[0.0073350710225801715,0.8313757584337543,-0.8095307954684084,2.441191686884721,0.5191713795290003,1.1534591903547016,-0.2989124112808717,-0.5128514186201779,-0.619712827067017,0.6956151804322931]
0.14228558260358093


In [113]:
result = lin_reg_model.evaluate(test_data)
print(result.rootMeanSquaredError)
print(result.r2)

8.731146318219317
0.052911772068891194


In [114]:
pred = lin_reg_model.transform(test_data.select('features'))
pred.show()

+--------------------+--------------------+
|            features|          prediction|
+--------------------+--------------------+
|(10,[0,1,2,3,4,5,...|  0.8366449600482407|
|(10,[0,1,2,3,4,5,...|  -1.475284763550391|
|(10,[0,1,2,3,4,5,...|  -2.680136401522974|
|(10,[0,1,2,3,4,5,...| -0.6346861633647207|
|(10,[0,1,2,3,4,5,...|  -0.976510689078842|
|(10,[0,1,2,3,4,5,...| -1.3820341784499024|
|(10,[0,1,2,3,4,5,...|  1.6690235491472345|
|(10,[0,1,2,3,4,5,...|  0.9290491838587673|
|(10,[0,1,2,3,4,5,...|  0.5789191740943999|
|(10,[0,1,2,3,4,5,...| -1.5735323090303452|
|(10,[0,1,2,3,4,5,...|  -2.415103670384772|
|(10,[0,1,2,3,4,5,...| -0.7680465872064948|
|(10,[0,1,2,3,4,5,...| -3.0756131143558623|
|(10,[0,1,2,3,4,5,...| 0.16437788237675283|
|(10,[0,1,2,3,4,5,...| -2.7189163477793263|
|(10,[0,1,2,3,4,5,...|  -4.603500793204407|
|(10,[0,1,2,3,4,5,...|-0.11085316828698039|
|(10,[0,1,2,3,4,5,...| 0.47959665597587847|
|(10,[0,1,2,3,4,5,...|  1.2616592891872596|
|(10,[0,1,2,3,4,5,...| -0.257288

## regularization
The elastic net penalty is a mixture of these two; alpha=0.5 tends to select the groups as in or out. 

If α =1, the elastic net performs much like the LASSO method. 
If α =0, the elastic net performs much like the Ridge method

LinearRegression(featuresCol="features", labelCol="label", predictionCol="prediction", maxIter=100,
regParam=0.0, elasticNetParam=0.0, tol=1e-6, fitIntercept=True, standardization=True, solver="auto",
weightCol=None, aggregationDepth=2)